<a href="https://colab.research.google.com/github/kevin6449/2026_speech/blob/main/intro_gemini_2_5_image_gen_ipynb_cn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 在 Vertex AI 上使用 Gemini 2.5 Flash Image（纳米香蕉 🍌）生成图像

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/kevin6449/2026_speech/blob/main/intro_gemini_2_5_image_gen_ipynb_cn.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> 在 Colab 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fkevin6449%2F2026_speech%2Frefs%2Fheads%2Fmain%2Fintro_gemini_2_5_image_gen_ipynb_cn.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> 在 Colab Enterprise 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/kevin6449/2026_speech/blob/main/intro_gemini_2_5_image_gen_ipynb_cn.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> 在 Vertex AI Workbench 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/kevin6449/2026_speech/blob/main/intro_gemini_2_5_image_gen_ipynb_cn.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> 在 GitHub 上查看
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>分享到：</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_image_gen.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_image_gen.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_image_gen.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_image_gen.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_image_gen.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| 作者 |
| --- |
| [Katie Nguyen](https://github.com/katiemn) |

## 概述

Gemini 2.5 Flash Image 是一种功能强大的通用多模态模型，提供最先进的图像生成和对话式图像编辑功能。这使您能够与 Gemini 对话并创建或编辑带有交错文本的图像。

在本教程中，您将学习如何使用 Google Gen AI SDK 在 Vertex AI 中使用 Gemini 2.5 Flash Image 来尝试以下场景：
  - 图像生成：
    - 文本到图像生成
    - 交错的图像和文本序列
  - 图像编辑：
    - 图像到图像（包含主题自定义和风格迁移）
    - 多轮图像编辑（带定位）
    - 使用多个参考图像进行编辑

## 开始使用

### 安装适用于 Python 的 Google Gen AI SDK

In [ ]:
%pip install --upgrade --quiet google-genai

### 验证您的笔记本环境（仅限 Colab）

如果您在 Google Colab 上运行此笔记本，请运行以下单元格以验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### 导入库

In [ ]:
from io import BytesIO

from IPython.display import Image, Markdown, display
from PIL import Image as PIL_Image
from google import genai
from google.genai.types import FinishReason, GenerateContentConfig, ImageConfig, Part
import matplotlib.image as img
import matplotlib.pyplot as plt
import requests

### 设置 Google Cloud 项目信息并创建客户端

要开始使用 Vertex AI，您必须拥有一个现有的 Google Cloud 项目并[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

详细了解[如何设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

In [ ]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[您的项目 ID]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "global"

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### 加载图像模型


In [ ]:
MODEL_ID = "gemini-2.5-flash-image"

## 图像生成

首先，您将向 Gemini 2.5 Flash Image 发送文本提示，描述您想要生成的图像。


### 文本到图像

在下面的单元格中，您将调用 `generate_content` 方法并传入以下参数：

  - `model`：您要使用的模型 ID。
  - `contents`：这是您的提示，在本例中是描述要生成图像的纯文本用户消息。
  - `config`：用于指定内容设置的配置。
    - `response_modalities`：要生成图像，您必须在 `response_modalities` 列表中包含 `IMAGE`。
    - `ImageConfig`：设置 `aspect_ratio`。有效比例为：“1:1”、“3:2”、“2:3”、“3:4”、“4:3”、“4:5”、“5:4”、“9:16”、“16:9”、“21:9”
    - `candidate_count`：要生成的候选数量。


所有生成的图像都包含 [SynthID 水印](https://deepmind.google/technologies/synthid/)，可以在 [Vertex AI Studio](https://cloud.google.com/generative-ai-studio?hl=en) 中的 Media Studio 进行验证。

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="一张关于飞行运动鞋的卡通信息图",
    config=GenerateContentConfig(
        response_modalities=["IMAGE"],
        image_config=ImageConfig(
            aspect_ratio="9:16",
        ),
        candidate_count=1,
    ),
)

# 如果未生成图像，则检查错误
if response.candidates[0].finish_reason != FinishReason.STOP:
    reason = response.candidates[0].finish_reason
    raise ValueError(f"提示内容错误：{reason}")


for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    if part.inline_data:
        display(Image(data=part.inline_data.data, width=500))

### 文本到图像和文本

除了生成图像，Gemini 还可以创建图像和文本的交错序列。

例如，您可以要求模型生成一份香蕉面包食谱，并附带显示烹饪过程不同阶段的图像。或者，您可以要求模型生成不同野花的图像，并附带标题和描述。

让我们通过提示 Gemini 2.5 Flash Image 创建一个关于制作花生酱和果酱三明治的教程来尝试交错文本和图像功能。

您会注意到，在提示中我们要求模型为每个步骤生成文本和图像。这鼓励模型创建交错的文本和图像。此外，请务必设置 `response_modalities=["TEXT", "IMAGE"]`，这样您将在响应中获得两种模态。

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="创建一个教程，分三个简单步骤解释如何制作花生酱和果酱三明治。对于每个步骤，提供带步骤编号的标题、说明，并生成一个图像来阐明内容。为每个图像标注步骤编号，但不要添加其他文字。",
    config=GenerateContentConfig(
        response_modalities=["TEXT", "IMAGE"],
        image_config=ImageConfig(
            aspect_ratio="4:3",
        ),
    ),
)

for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    if part.inline_data:
        display(Image(data=part.inline_data.data, width=500))

## 图像编辑

Gemini 2.5 Flash Image 可以从多个参考图像生成图像到图像的输出。这对于确保角色一致性、生成徽标、风格迁移以及插入或移除对象等任务非常有用。

### 主题自定义

让我们通过要求 Gemini 2.5 Flash Image 以铅笔素描风格创建这只狗的图像来尝试主题自定义示例。


#### 下载狗的图像

以下示例使用 Cloud Storage 中的图像。如果您希望使用不同的图像，可以更改 `wget` 命令中的 URL，或者，如果您有本地文件，请在后续步骤中更新 `subject_image` 变量。

In [ ]:
!wget https://storage.googleapis.com/cloud-samples-data/generative-ai/image/dog-1.jpg

In [ ]:
subject_image = "dog-1.jpg"  # @param {type: 'string'}

# 显示图像
fig, axis = plt.subplots(1, 1, figsize=(6, 12))
axis.imshow(img.imread(subject_image))
axis.axis("off")
plt.show()

#### 发送请求

由于您在请求中包含了来自本地图像的数据，因此需要将 `Part.from_bytes` 包含到请求的 `contents` 中。

In [ ]:
with open(subject_image, "rb") as f:
    image = f.read()

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_bytes(
            data=image,
            mime_type="image/jpeg",
        ),
        "创建一个戴着牛仔帽的狗的铅笔素描图像，背景为西部主题。",
    ],
    config=GenerateContentConfig(
        response_modalities=["IMAGE"],
        image_config=ImageConfig(
            aspect_ratio="1:1",
        ),
        candidate_count=1,
    ),
)

for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    if part.inline_data:
        display(Image(data=part.inline_data.data, width=500))

### 风格迁移

在下一个示例中，您将使用客厅的风格来重新构想一个相同风格的厨房。

#### 下载客厅图像

同样，以下示例使用 Cloud Storage 中的图像。如果您希望使用不同的图像，可以更改 `wget` 命令中的 URL，或者，如果您有本地文件，请在后续步骤中更新 `style_image` 变量。

In [ ]:
!wget https://storage.googleapis.com/cloud-samples-data/generative-ai/image/living-room.png

In [ ]:
style_image = "living-room.png"  # @param {type: 'string'}

# 显示图像
fig, axis = plt.subplots(1, 1, figsize=(6, 12))
axis.imshow(img.imread(style_image))
axis.axis("off")
plt.show()

#### 发送请求

In [ ]:
with open(style_image, "rb") as f:
    image = f.read()

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_bytes(
            data=image,
            mime_type="image/png",
        ),
        "使用这个客厅的理念、颜色和主题，生成一个具有相同美学的厨房和餐厅。",
    ],
    config=GenerateContentConfig(
        response_modalities=["IMAGE"],
        image_config=ImageConfig(
            aspect_ratio="21:9",
        ),
        candidate_count=1,
    ),
)

for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    if part.inline_data:
        display(Image(data=part.inline_data.data, width=500))

### 多轮图像编辑

在下一节中，您将提供一个起始图像，并通过与 Gemini 2.5 Flash Image 对话来迭代地改变图像的某些方面。


在下一个示例中，您将不使用本地图像，而是使用存储在 Google Cloud Storage 中的图像。运行下一步以查看香水瓶的起始图像。如果您想使用 Cloud Storage 中的不同图像，请替换下面的 `perfume_url` 和 `perfume_uri`。

In [ ]:
perfume_url = (
    "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/perfume.jpg"
)
perfume_uri = "gs://cloud-samples-data/generative-ai/image/perfume.jpg"


# 显示图像
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
perfume_image = PIL_Image.open(BytesIO(requests.get(perfume_url).content))
axes[0].imshow(perfume_image)
for i, ax in enumerate(axes):
    ax.axis("off")
plt.show()

#### 开始聊天

在下一步中，您将开始一个聊天，以便通过与 Gemini 对话来持续编辑图像。由于您现在使用存储在 Cloud Storage 中的参考图像，因此您将在 `message` 内容中使用 `Part.from_uri`。

In [ ]:
chat = client.chats.create(model=MODEL_ID)

response = chat.send_message(
    message=[
        Part.from_uri(
            file_uri=perfume_uri,
            mime_type="image/jpeg",
        ),
        "将香水颜色改为淡紫色",
    ],
    config=GenerateContentConfig(
        response_modalities=["IMAGE"],
        image_config=ImageConfig(
            aspect_ratio="3:2",
        ),
    ),
)

data = perfume_uri
for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    if part.inline_data:
        display(Image(data=part.inline_data.data, width=500))
        data = part.inline_data.data

现在，您将在现有聊天中包含之前的图像数据以及新的文本提示，以更新之前生成的图像。这次，您将要求在香水瓶上写一个单词，由于 Gemini 能够处理不同的语言，我们来要求用法语写这个单词。

In [ ]:
response = chat.send_message(
    message=[
        Part.from_bytes(
            data=data,
            mime_type="image/jpeg",
        ),
        "在香水瓶上以精致的白色草书字体刻上法文单词 'flowers'",
    ],
    config=GenerateContentConfig(
        response_modalities=["IMAGE"],
        image_config=ImageConfig(
            aspect_ratio="3:2",
        ),
    ),
)

for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    if part.inline_data:
        display(Image(data=part.inline_data.data, width=500))

### 多个参考图像

在使用 Gemini 2.5 Flash Image 编辑图像时，您还可以提供多个输入图像来创建新的图像。在下一个示例中，您将使用一张女性和一张手提箱的图像来提示 Gemini。然后，您将要求 Gemini 组合这些图像中的对象以创建一个新图像。您还将要求 Gemini 提供与图像相伴的文本。


运行以下单元格以可视化存储在 Cloud Storage 中的起始图像。

In [ ]:
person_url = (
    "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/woman.jpg"
)
suitcase_url = (
    "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/suitcase.png"
)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
person_image = PIL_Image.open(BytesIO(requests.get(person_url).content))
axes[0].imshow(person_image)
suitcase_image = PIL_Image.open(BytesIO(requests.get(suitcase_url).content))
axes[1].imshow(suitcase_image)
for i, ax in enumerate(axes):
    ax.axis("off")
plt.show()

现在，您将发送请求。与之前的图像编辑调用唯一的区别是，您将根据参考图像的数量提供多个 `Part.from_uri` 实例。

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri="gs://cloud-samples-data/generative-ai/image/suitcase.png",
            mime_type="image/png",
        ),
        Part.from_uri(
            file_uri="gs://cloud-samples-data/generative-ai/image/woman.jpg",
            mime_type="image/jpeg",
        ),
        "生成一张女性在机场拉着手提箱的图像。另外，为这张图像写一个适合社交媒体发布的简短标题。",
    ],
    config=GenerateContentConfig(
        response_modalities=["TEXT", "IMAGE"],
        image_config=ImageConfig(
            aspect_ratio="9:16",
        ),
        candidate_count=1,
    ),
)


for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    if part.inline_data:
        display(Image(data=part.inline_data.data, width=500))